In [2]:
import numpy as np
import pandas as pd
import yfinance as yf #pip install yfinance
tick = yf.Ticker('^GSPC')
hist = tick.history(period="max", rounding=True)
#hist = hist[:'2019-10-07']
hist = hist[-1000:]
h = hist.Close.tolist()
display(h)

[2172.47,
 2169.04,
 2180.38,
 2176.12,
 2170.95,
 2170.86,
 2179.98,
 2186.48,
 2186.16,
 2181.3,
 2127.81,
 2159.04,
 2127.02,
 2125.77,
 2147.26,
 2139.16,
 2139.12,
 2139.76,
 2163.12,
 2177.18,
 2164.69,
 2146.1,
 2159.93,
 2171.37,
 2151.13,
 2168.27,
 2161.2,
 2150.49,
 2159.73,
 2160.77,
 2153.74,
 2163.66,
 2136.73,
 2139.18,
 2132.55,
 2132.98,
 2126.5,
 2139.6,
 2144.29,
 2141.34,
 2141.16,
 2151.33,
 2143.16,
 2139.43,
 2133.04,
 2126.41,
 2126.15,
 2111.72,
 2097.94,
 2088.66,
 2085.18,
 2131.52,
 2139.56,
 2163.26,
 2167.48,
 2164.45,
 2164.2,
 2180.39,
 2176.94,
 2187.12,
 2181.9,
 2198.18,
 2202.94,
 2204.72,
 2213.35,
 2201.72,
 2204.66,
 2198.81,
 2191.08,
 2191.95,
 2204.71,
 2212.23,
 2241.35,
 2246.19,
 2259.53,
 2256.96,
 2271.72,
 2253.28,
 2262.03,
 2258.07,
 2262.53,
 2270.76,
 2265.18,
 2260.96,
 2263.79,
 2268.88,
 2249.92,
 2249.26,
 2238.83,
 2257.83,
 2270.75,
 2269.0,
 2276.98,
 2268.9,
 2268.9,
 2275.32,
 2270.44,
 2274.64,
 2267.89,
 2271.89,
 2263.69,


In [3]:
####   All peaks and troughs naive method
minimaIdxs = np.flatnonzero(
 hist.Close.rolling(window=3, min_periods=1, center=True).aggregate(
   lambda x: len(x) == 3 and x[0] > x[1] and x[2] > x[1])).tolist()
maximaIdxs = np.flatnonzero(
 hist.Close.rolling(window=3, min_periods=1, center=True).aggregate(
   lambda x: len(x) == 3 and x[0] < x[1] and x[2] < x[1])).tolist()

In [4]:
###  All peaks and troughs handling consecutive duplicates method

hs = hist.Close.loc[hist.Close.shift(-1) != hist.Close]
x = hs.rolling(window=3, center=True).aggregate(lambda x: x[0] > x[1] and x[2] > x[1])
minimaIdxs = [hist.index.get_loc(y) for y in x[x == 1].index]
x = hs.rolling(window=3, center=True).aggregate(lambda x: x[0] < x[1] and x[2] < x[1])
maximaIdxs = [hist.index.get_loc(y) for y in x[x == 1].index]



In [10]:
#### Numerical Differentiation method
from findiff import FinDiff #pip install findiff
dx = 1 #1 day interval
d_dx = FinDiff(0, dx, 1)
d2_dx2 = FinDiff(0, dx, 2)
clarr = np.asarray(hist.Close)
mom = d_dx(clarr)
momacc = d2_dx2(clarr)

def get_extrema(isMin):
    return [x for x in range(len(mom))
    if (momacc[x] > 0 if isMin else momacc[x] < 0) and
      (mom[x] == 0 or #slope is 0
        (x != len(mom) - 1 and #check next day
          (mom[x] > 0 and mom[x+1] < 0 and
           h[x] >= h[x+1] or
           mom[x] < 0 and mom[x+1] > 0 and
           h[x] <= h[x+1]) or
         x != 0 and #check prior day
          (mom[x-1] > 0 and mom[x] < 0 and
           h[x-1] < h[x] or
           mom[x-1] < 0 and mom[x] > 0 and
           h[x-1] > h[x])))]
minimaIdxs, maximaIdxs = get_extrema(True), get_extrema(False)

display(type(minimaIdxs))
display(minimaIdxs)

list

[1,
 5,
 13,
 16,
 21,
 24,
 27,
 30,
 36,
 40,
 50,
 56,
 68,
 77,
 83,
 88,
 93,
 100,
 108,
 112,
 123,
 127,
 133,
 137,
 142,
 146,
 153,
 159,
 162,
 169,
 172,
 176,
 179,
 182,
 191,
 195,
 199,
 202,
 207,
 210,
 212,
 216,
 228,
 233,
 236,
 241,
 247,
 251,
 258,
 261,
 265,
 272,
 282,
 285,
 294,
 297,
 306,
 309,
 311,
 323,
 329,
 333,
 336,
 339,
 363,
 366,
 374,
 380,
 390,
 396,
 399,
 401,
 405,
 417,
 424,
 432,
 435,
 441,
 452,
 462,
 466,
 471,
 478,
 484,
 494,
 496,
 502,
 512,
 518,
 525,
 536,
 545,
 548,
 552,
 560,
 566,
 575,
 586,
 592,
 599,
 604,
 609,
 616,
 625,
 630,
 636,
 644,
 647,
 649,
 658,
 664,
 669,
 674,
 679,
 681,
 686,
 695,
 702,
 704,
 712,
 719,
 728,
 732,
 739,
 744,
 746,
 753,
 759,
 768,
 774,
 780,
 784,
 792,
 794,
 804,
 808,
 816,
 823,
 828,
 834,
 841,
 844,
 855,
 859,
 863,
 868,
 874,
 882,
 893,
 898,
 902,
 905,
 914,
 918,
 926,
 934,
 940,
 954,
 960,
 965,
 971,
 978,
 984,
 996,
 999]

In [9]:
def get_bestfit(pts):
    xbar, ybar = [sum(x) / len (x) for x in zip(pts)]
    def subcalc(x, y):
        tx, ty = x - xbar, y - ybar
        return tx * ty, tx * tx, x * x
        (xy, xs, xx) = [sum(q) for q in zip(*[subcalc(x, y) for x, y in pts])]
        m = xy / xs
        b = ybar - m * xbar
        ys = sum([np.square(y - (m * x + b)) for x, y in pts])
        ser = np.sqrt(ys / ((len(pts) - 2) * xs))
        return m, b, ys, ser, ser * np.sqrt(xx / len(pts))

m, b, ys, ser, ser = get_bestfit(minimaIdxs)
    

ValueError: too many values to unpack (expected 2)